In [ ]:
!pip install poetry

In [ ]:

!pip install pyspark
!pip install pandas

In [2]:
from typing import Set
from typing import Tuple

import pandas as pd
from pandas import DataFrame as DaFe

In [3]:
"""EXERCISE 1
1. ¿Cuántos registros hay?"""

def get_row_quantity(to_count: DaFe) -> int:
    return len(to_count.index)

# this value can be decreased to use less ram, but slower
# if increased, faster but more ram is used,
# as working with large sets of data,
# lower value is better as possible.
CHUNK_SIZE = 2500

def get_row_number_for_big_csv(csv_location: str) -> int:
    """optimized way to get number of rows in a big csv file"""
    # this line will delete non/sense rows

    # TODO save last chunk index processed if error to help resume
    counter = 0
    # never use this my_csv = pd.read_csv('all_data.csv')
    # because if csv is 20gb then ram used is 20gb
    with pd.read_csv(csv_location, chunksize = CHUNK_SIZE) as file_reader:
        # don't need to count because each page has same size
        for chunk in file_reader:
            counter += CHUNK_SIZE
        # get exact row count for last chunk, it may be <= CHUNK_SIZE
        counter -= CHUNK_SIZE - get_row_quantity(chunk)
    return counter

# estimated running time = 5 minutes, unlimited ram
# for 20 gb file (RYZEN 3 3200g similar to i3 10th gen)
print(get_row_number_for_big_csv('all_data.csv')) # answer is 62530715

62530715


In [3]:
"""EXERCISE 1
2. ¿Cuántas categorías?"""
# aqui supongo que las categorias son por cliente
CATEGORY_COLUMN_NAME = 'categoria'

# this value can be decreased to use less ram, but slower
# if increased, faster but more ram is used,
# as working with large sets of data,
# lower value is better as possible.
CHUNK_SIZE = 2500

def get_unique_column_values_quantity(csv_location: str, column_name:str) -> int:
    """optimized way to get unique column_values in a big csv file"""
    ...

print(get_unique_column_values_quantity('all_data.csv',CATEGORY_COLUMN_NAME))

None


In [ ]:
"""EXERCISE 1
3. ¿Cuántas cadenas comerciales están siendo monitoreadas (y, por lo tanto, reportadas
en esa base de datos)?"""

# this value can be decreased to use less ram, but slower
# if increased, faster but more ram is used,
# as working with large sets of data,
# lower value is better as possible.
CHUNK_SIZE = 250
CADENA_COMERCIAL_COLUMN = "cadenaComercial"


def get_higher_appearances_in_column(column_name: str, csv_location: str) -> int:
    """optimized way to get total number of unique column values in a big csv file"""
    output: Set[str] = set()
    # never use pd.read_csv('all_data.csv')
    # because if csv is 20gb then ram used is 20gb
    with pd.read_csv(csv_location, chunksize=CHUNK_SIZE) as file_reader:
        for chunk in file_reader:
            # silently drop nan values & duplicates
            chunk[column_name].dropna().drop_duplicates(inplace=True)
            # perform merge to obtain unique values
            output.update(chunk)
    return len(output)


# estimated running time = 5 minutes for 20 gb file unlimited ram
# (RYZEN 3 3200g similar to i3 10th gen)
ANSWER = get_higher_appearances_in_column(
    CADENA_COMERCIAL_COLUMN, csv_location="all_data.csv"
)  # ANSWER = 705

Empty DataFrame
Columns: [estado, producto, cantidad]
Index: []


RuntimeError: No active exception to reraise

In [ ]:
"""EXERCISE 1
5. ¿Cuál es la cadena comercial con mayor variedad de productos monitoreados?"""
# mayor variedad significa la mayor cantidad de productos unicos/diferentes

# this value can be decreased to use less ram, but slower
# if increased, faster but more ram is used,
# as working with large sets of data,
# lower value is better as possible.
CHUNK_SIZE = 2500
PRODUCT_COLUMN = 'producto'
CADENA_COMERCIAL_COLUMN = 'cadenaComercial'
QUANTITY_NEW_COLUMN = 'cantidad'

def obtain_unique_values_from_big_csv(csv_location, parent_column:str, children_column:str) -> DaFe:
    output = DaFe(columns =[parent_column,children_column])
    with pd.read_csv(csv_location, chunksize = CHUNK_SIZE) as file_reader:
        for chunk in file_reader:
            # shrink to perform calculations
            chunk = chunk[[parent_column,children_column]]
            chunk.drop_duplicates(inplace=True)
            output = pd.merge(output, chunk, 'outer',
                     on=[parent_column, children_column])\
                    .set_index([parent_column, children_column])\
                    .reset_index()
    # assert output == output.drop_duplicates()
    output = output.value_counts(subset=parent_column)
    output.columns = [parent_column,QUANTITY_NEW_COLUMN]
    return output


def get_higher_unique_values(csv_location:str, parent_column:str,children_column:str) -> Tuple[str, int]:
    """get name and quantity of higher parent_column with most children_column unique values"""
    unique_repeated_values = obtain_unique_values_from_big_csv(csv_location,parent_column,children_column)
    unique_repeated_values.to_csv('5.- unique_values.csv',index=False)
    unique_repeated_values.sort_values(axis=0, ascending=False, inplace=True)
    region_quantity_relationship = unique_repeated_values.head(1).to_dict()

    the_parent = list(region_quantity_relationship).pop()
    quantity = region_quantity_relationship[the_parent]
    return the_parent, quantity

# estimated running time = 10 minutes for 20 gb file unlimited ram
# (RYZEN 3 3200g similar to i3 10th gen)
print(get_higher_unique_values('all_data.csv',CADENA_COMERCIAL_COLUMN,PRODUCT_COLUMN))
# answer

In [ ]:
"""EXERCISE 1
6. Encuentra algún dato curioso en los datos y comunícalo en un slide de powerpoint."""
# cual es el producto mas caro
# obtener producto,cadenaComercial,precio,marca,presentacion,categoria
# (mejor obtener todo y limpiar la fila)